<br>
<br>
<br>
<a id="intro"></a>


# Index notebook tools 
This notebook includes all the data and tools for producing the content of the index notebook.



The notebook is divided into the following parts:
- [Import Python modules](#import_modules)
- [Read JSON-files](#read_json)
- [Format text as HTML](#format_txt_as_html)
- [Create ipywidget GridBox for storing info of a single notebook](#create_notebook_gridbox)
- [Create ipywidget Accordion of notebooks](#create_notebooks_accordion)
- [Create parent ipywidget Accordion](#create_parent_accordion)

<br>
<br>

<a id='import_modules'></a>
<br>
<br>


## Import modules

In [ ]:
# Standard library imports.
from collections import OrderedDict
import json

# Related third party imports.
from ipywidgets import Accordion, GridBox, HTML, Image, Layout

<a id='read_json'></a>
<div style="text-align: right"><a href="#intro">Back to top</a></div>
<br>
<br>

## Read JSON-files

In [ ]:
def read_json(path=None):
    """Read json file and load content to dictionary."""
    with open(file=path, mode='r') as json_handle:
        return json.load(json_handle, object_pairs_hook=OrderedDict)


# Read static notebooks' content.
index_notebooks = read_json(path='./icos_jupyter_notebooks/index-tools/json/index_notebooks.json')

<a id='format_txt_as_html'></a>
<div style="text-align: right"><a href="#intro">Back to top</a></div>

<br>
<br>

## Format text as HTML

In [ ]:
def get_html_email(link_list):
    """Generate an HTML email-link string."""
    html_email_str = str()
    mail_list = list()
    if isinstance(link_list, list):
        # Check if list-items are email-addresses
        if all('@' in email for email in link_list):
            for email in link_list:
                # Create HTML email-links for every list item.
                mail_link = (
                    f'<a '
                    f'href="mailto:{email}"'
                    # Links created using the ipywidgets HTML class do
                    # not automatically get the correct styling, so we
                    # set the correct values to its attributes.
                    f'style="text-decoration:underline;color:blue;" '
                    f'>{email}</a>')
                mail_list.append(mail_link)
            # Concatenate list of e-mails to string.
            html_email_str = ', '. join(mail_list)
    return html_email_str

<a id='create_notebook_gridbox'></a>
<div style="text-align: right"><a href="#intro">Back to top</a></div>
<br>
<br>

## Create ipywidget GridBox for storing info of a single notebook

In [ ]:
def create_gridbox(notebook_info):
    """Create a Grid-Box of notebooks' information."""
    # Note to future developer: In this current version, the only way
    # to correctly open a link of a notebook (from Python) to a new
    # Jupyter-Lab tab, is if the href tag is formatted like the one
    # below. In notebook-markdown the case is much easier since Jupyter
    # understands between local and non-local paths.
    formatted_arguments = (
        f'{{&quot;path&quot;:&quot;{notebook_info["relative_path"]}&quot;, '
        f'&quot;id&quot;:&quot;&quot;}}')
    href_tag = ('<a '
                'href="%s" '
                # Links created using the ipywidgets HTML class do not
                # automatically get the correct styling, so we set the
                # correct values to its attributes.
                'style="text-decoration:underline;color:blue;" '
                # Attributes used to open a new Jupyter-Lab tab instead
                # of opening a new browser tab.
                'data-commandlinker-command="rendermime:handle-local-link" '
                'data-commandlinker-args="%s">%s</a>')\
               % (notebook_info['relative_path'], formatted_arguments, notebook_info['title'])
    git_issues_url = 'https://github.com/ICOS-Carbon-Portal/jupyter/issues'
    thumbnail = None
    if notebook_info['thumbnail']:
        thumbnail = Image(value=open(
            file=notebook_info['thumbnail'], mode='rb').read())
    else:
        thumbnail = HTML(value='No thumbnail')
    # Store notebook information to ipywidget-gridbox.
    notebook_grid_box = GridBox(
        children=[
            HTML(value='<b>Title: </b> '),
            HTML(value=f'<b>{href_tag}</b>'),
            HTML(value='<b>Description: </b> '),
            HTML(value=
                 # Reduce space between lines and description's width.
                 f'<p style="line-height: 1em;">'
                 f'{notebook_info["description"]}'
                 f'</p>'),
            HTML(value='<b>Contact: </b> '),
            HTML(value=get_html_email(notebook_info['contact'])),
            HTML(value='<b>GitHub: </b> '),
            HTML(value=
                 f'<a href="{git_issues_url}" '
                 # Links created using the ipywidgets HTML class do
                 # not automatically get the correct styling, so we
                 # set the correct values to its attributes.
                 f'style="text-decoration:underline;color:blue;" '
                 f'target="_blank">Click here to report an issue</a>'),
            HTML(value='<b>Thumbnail: </b> '),
            thumbnail],
        # Set width of entire Grid-Box.
        layout=Layout(width = '100%',
                      # Set width per column.
                      grid_template_columns = '100px auto',
                      # Set height per row.
                      grid_template_rows = 'auto auto auto auto auto',
                      # Set gap between rows and columns.
                      grid_gap = '0px 0px',
                      # Set padding top, right, bottom, left.
                      padding = '0px 0px 0px 0px'))
    # Return Grid-Box.
    return notebook_grid_box

<a id='create_notebooks_accordion'></a>
<div style="text-align: right"><a href="#intro">Back to top</a></div>
<br>
<br>

## Create ipywidget Accordion of notebooks

In [ ]:
def create_notebooks_accordion(category: str = None) -> str:
    """Create a notebook accordion."""
    l_notebooks = list()
    l_titles = list()
    # Iterate all notebooks of the same category.
    for notebook in index_notebooks['notebook_mapping'][category]:
        l_notebooks.append(create_gridbox(notebook))
        l_titles.append(notebook['title'])
    accordion = Accordion(children=l_notebooks, titles=l_titles)
    return accordion



<a id='create_parent_accordion'></a>
<div style="text-align: right"><a href="#intro">Back to top</a></div>
<br>
<br>

## Create parent ipywidget Accordion

In [ ]:
def main_accordion() -> Accordion:
    cat_1_children = list()
    cat_1_titles = list()
    # Loop through all notebook main-categories.
    for cat_1, info_1 in index_notebooks['category_mapping'].items():
        cat_2_children = list()
        cat_2_titles = list()
        cat_1_description = None
        # Loop through all notebook sub-categories.
        for cat_2, info_2 in info_1.items():
            if cat_2 == 'description':
                # The description belongs to the main category, but it
                # is displayed with the main category's children.
                cat_1_description = HTML(value=info_1['description'])
            else:
                cat_3_children = list()
                cat_3_titles = list()
                cat_2_description = None
                # Loop through all notebook sub-sub-categories.
                for cat_3, info_3 in info_2.items():
                    if cat_3 == 'description':
                        # The description belongs to a sub-category,
                        # but it is displayed with the sub-category's
                        # children.
                        cat_2_description = HTML(value=info_2['description'])
                    else:
                        # The description belongs to a
                        # sub-sub-category, but it is displayed with
                        # the sub-sub-category's children.
                        cat_3_description = HTML(value=info_3['description'])
                        # Construct the category key to be used with
                        # 'notebook_mapping' entry in
                        # index_notebooks.json file.
                        category = ', '.join(filter(None, [cat_1,
                                                           cat_2,
                                                           cat_3]))
                        # Found notebooks to append to widget.
                        # No need to search for category in
                        # ['notebook_mapping'].keys(), since this is
                        # the end-point and notebooks must appear here.
                        notebooks_accordion = \
                            create_notebooks_accordion(category)
                        cat_3_children.append(GridBox(
                            children=[cat_3_description, notebooks_accordion]
                        ))
                        # Titles to be used with parent accordion.
                        cat_3_titles.append(cat_3)
                # Construct the category key to be used with
                # 'notebook_mapping' entry in index_notebooks.json
                # file.
                category = ', '.join(filter(None, [cat_1, cat_2]))
                # Found notebooks to append to widget.
                if category in index_notebooks['notebook_mapping'].keys():
                    notebooks_accordion = \
                        create_notebooks_accordion(category)
                    cat_2_children.append(GridBox(
                        children=[cat_2_description, notebooks_accordion]
                    ))
                # No notebooks under the specified category.
                # Append sub-sub-categories.
                else:
                    cat_3_accordion = Accordion(children=cat_3_children,
                                                titles=cat_3_titles)
                    cat_2_children.append(GridBox(
                        children=[cat_2_description, cat_3_accordion]
                    ))
                # Titles to be used with parent accordion.
                cat_2_titles.append(cat_2)
        # Construct the category key to be used with 'notebook_mapping'
        # entry in index_notebooks.json file.
        category = cat_1
        # Found notebooks to append to widget.
        if category in index_notebooks['notebook_mapping'].keys():
            notebooks_accordion = create_notebooks_accordion(category)
            cat_1_children.append(GridBox(
                children=[cat_1_description, notebooks_accordion]
            ))
        # No notebooks under the specified category.
        # Append sub-categories.
        else:
            cat_2_accordion = Accordion(children=cat_2_children,
                                        titles=cat_2_titles)
            cat_1_children.append(GridBox(children=[cat_1_description,
                                                    cat_2_accordion]))
        # Titles to be used with parent accordion.
        cat_1_titles.append(cat_1)
    parent_accordion = Accordion(children=cat_1_children,
                                 titles=cat_1_titles)
    return parent_accordion

<br>
<br>
<div style="text-align: right"><a href="#intro">Back to top</a></div>
<br>
<br>